# Apriori Algorithm

Algorithm to "predict" and add Association Rules to products. Regularly used in "Shop Carts"

![Amazon](./amazon.png)

_Amazon Example_

Fallowing [this](https://www.kaggle.com/ozlemilgun/market-basket-analysis-with-apriori-algorithm/notebook) Kaggle Post.

In [1]:
# Installation Process. Ignore if already have
%%capture capt
!pip install mlxtend
!pip install pandas
!pip install numpy
!pip install openpyxl

UsageError: Line magic function `%%capture` not found.


In [2]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori, association_rules

In [3]:
# Configure
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)

In [4]:
base = pd.read_excel('./online_retail_II.xlsx', sheet_name='Year 2010-2011')

In [5]:
base.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.550,17850.000,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.390,17850.000,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.750,17850.000,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.390,17850.000,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.390,17850.000,United Kingdom


In [6]:
base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541910 entries, 0 to 541909
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      541910 non-null  object        
 1   StockCode    541910 non-null  object        
 2   Description  540456 non-null  object        
 3   Quantity     541910 non-null  int64         
 4   InvoiceDate  541910 non-null  datetime64[ns]
 5   Price        541910 non-null  float64       
 6   Customer ID  406830 non-null  float64       
 7   Country      541910 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [7]:
base.isna().sum() 

Invoice             0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
Price               0
Customer ID    135080
Country             0
dtype: int64

In [8]:
base.dropna(inplace=True)

In [9]:
base_des = base.describe().T
base_des

,count,mean,std,min,25%,50%,75%,max
Quantity,406830.000,12.061,248.693,-80995.000,2.000,5.000,12.000,80995.000
Price,406830.000,3.461,69.315,0.000,1.250,1.950,3.750,38970.000
Customer ID,406830.000,15287.684,1713.603,12346.000,13953.000,15152.000,16791.000,18287.000


Since the invoices above are cancelled, they should be removed from Apriori Algorith.

In [10]:
# Canceled Transactions
df_Invoice = pd.DataFrame({"Invoice":[row for row in base["Invoice"].values if "C"  not in str(row)]})
df_Invoice.head()
df_Invoice = df_Invoice.drop_duplicates("Invoice")

# Merging dataframes to remove cancelled transactions
base = base.merge(df_Invoice, on = "Invoice")

In [11]:
# Outliers

def outlier_thresholds(dataframe, variable):
    """
    Calculates outilers based on first and third quantile.
    """
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit

def replace_with_thresholds(dataframe, variable):
    """
    Replace outliers with the thresholds.
    """
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit

In [12]:
# Get types
base.dtypes

Invoice                object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
Price                 float64
Customer ID           float64
Country                object
dtype: object

In [13]:
# Get numerical cols
num_cols = [col for col in base.columns if base[col].dtypes in ["int64","float64"] and "ID" not in col]

In [14]:
# Removing outliers from numerical columns base
for col in num_cols:
    replace_with_thresholds(base, col)

In [15]:
base.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,397925.000,11.834,25.534,1.000,2.000,6.000,12.000,298.500
Price,397925.000,2.893,3.227,0.000,1.250,1.950,3.750,37.060
Customer ID,397925.000,15294.309,1713.173,12346.000,13969.000,15159.000,16795.000,18287.000


In [16]:
# Comparing Prices
print(base_des["mean"]['Price'])
print(base.describe().T["mean"]['Price'])

3.4605067571221397
2.893200738832695


In [17]:
base.shape

(397925, 8)

In [18]:
# Study: The outliers are gone, but that does't have anything to do with too much quantity = prices too high?
base = base[base["Quantity"] > 0] 
base = base[base["Price"] > 0]

In [19]:
base.shape

(397885, 8)

In [20]:
print(base.Description.nunique())
base.StockCode.nunique()

3877


3665

The values above should be equal, Since Description == 'Unique Product' == StockCode 

In [21]:
# 1st Step: If one Product has more than one Stock Code
df_product = base[["Description","StockCode"]].drop_duplicates()
df_product = df_product.groupby(["Description"]).agg({"StockCode":"count"}).reset_index()
df_product.sort_values("StockCode", ascending=False).head()

,Description,StockCode
2014,"METAL SIGN,CUPCAKE SINGLE HOOK",3
2471,PINK FLOWERS RABBIT EASTER,2
1077,EAU DE NILE JEWELLED PHOTOFRAME,2
2460,PINK FAIRY CAKE CUSHION COVER,2
3678,WHITE BAMBOO RIBS LAMPSHADE,2


In [22]:
df_product.rename(columns={'StockCode':'StockCode_Count'},inplace=True)
# Products with more than one StockCode are "garbage"
df_product = df_product[df_product["StockCode_Count"]>1]

In [23]:
base = base[~base["Description"].isin(df_product["Description"])]

In [24]:
print(base.StockCode.nunique())
print(base.Description.nunique())

3630
3858


In [25]:
# 2nd Step: If Stock Code represents more than one product
df_product = base[["Description","StockCode"]].drop_duplicates()
df_product = df_product.groupby(["StockCode"]).agg({"Description":"count"}).reset_index()
df_product.rename(columns={'Description':'Description_Count'},inplace=True)

In [26]:
df_product = df_product.sort_values("Description_Count", ascending=False)
df_product.head()

,StockCode,Description_Count
1990,23236,4
1950,23196,4
2104,23366,3
1886,23131,3
1994,23240,3


In [27]:
df_product = df_product[df_product["Description_Count"] > 1] 

df_product.head()

,StockCode,Description_Count
1990,23236,4
1950,23196,4
2104,23366,3
1886,23131,3
1994,23240,3


In [28]:
base = base[~base["StockCode"].isin(df_product["StockCode"])]

In [29]:
print(base.StockCode.nunique())
print(base.Description.nunique())

3420
3420


In [30]:
base.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8.000,2010-12-01 08:26:00,2.750,17850.000,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6.000,2010-12-01 08:26:00,3.390,17850.000,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6.000,2010-12-01 08:26:00,3.390,17850.000,United Kingdom
5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2.000,2010-12-01 08:26:00,7.650,17850.000,United Kingdom
6,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6.000,2010-12-01 08:26:00,4.250,17850.000,United Kingdom


Now every Stock represents one Product and every product represents onde Stock

In [31]:
# Every POST in StockCode shows the postage cost. Since its not useful, we will remove it
base = base[~base["StockCode"].str.contains("POST", na=False)]

In [33]:
base.shape

(357405, 8)

In [35]:
np.unique(base.Country)

array(['Australia', 'Austria', 'Bahrain', 'Belgium', 'Brazil', 'Canada',
       'Channel Islands', 'Cyprus', 'Czech Republic', 'Denmark', 'EIRE',
       'European Community', 'Finland', 'France', 'Germany', 'Greece',
       'Iceland', 'Israel', 'Italy', 'Japan', 'Lebanon', 'Lithuania',
       'Malta', 'Netherlands', 'Norway', 'Poland', 'Portugal', 'RSA',
       'Saudi Arabia', 'Singapore', 'Spain', 'Sweden', 'Switzerland',
       'USA', 'United Arab Emirates', 'United Kingdom', 'Unspecified'],
      dtype=object)

Let's start by analyzing the Germany Data.

In [38]:
base_german = base[base.Country == 'Germany']
base_german.shape

(7843, 8)

## Create an Invoice-Product Matrix for ARL Data-structure

In order to use apriori algorithm we need a properly data-structure. 

In [39]:
def create_invoice_product_df(dataframe, id=False):
    if id:
        return dataframe.groupby(['Invoice', "StockCode"])['Quantity'].sum().unstack().fillna(0). \
            applymap(lambda x: 1 if x > 0 else 0)
    else:
        return dataframe.groupby(['Invoice', 'Description'])['Quantity'].sum().unstack().fillna(0). \
            applymap(lambda x: 1 if x > 0 else 0)